In [ ]:
import geopandas as gpd
import pandas as pd
import plotly.express as px

# Load election data
data_dir = "../data/"
income = pd.read_csv(data_dir + "income_data.csv", sep=',')
income

In [ ]:
# Extract Barcelona data
income_bcn = income[income['Municipalities'] == '08019 Barcelona']
income_bcn


In [ ]:
# Convert the 'Total' column to numeric using .loc
income_bcn.loc[:, 'Total'] = income_bcn['Total'].astype(float)

income_bcn.loc[:, 'Districts'] = income_bcn['Districts'].astype(float)

# Check the data type again
total_data_type = income_bcn['Total'].dtype

# Display the result
print("Data type of 'Total' column after conversion:", total_data_type)


In [ ]:
#income_bcn.loc[:, 'Total'] = income_bcn['Total'].astype(float)
#total_data_type = income_bcn['Total'].dtype
print(total_data_type)

In [ ]:
#income_bcn.loc[:, 'Districts'] = income_bcn['Districts'].astype(float)
#distr_data_type = income_bcn['Districts'].dtype
print(distr_data_type)

In [ ]:
# Add up votes for each district of Barcelona
income_bcn_district = income_bcn.groupby('Districts')['Total'].mean().reset_index().sort_values(by='Districts')
income_bcn_district

In [ ]:
# Read Barcelona's districts map
bcn = gpd.read_file(data_dir + "districts_BCN.geojson")
bcn.head()
# Select only the columns 'NOM' and 'geometry' from the GeoDataFrame
bcn = bcn[['NOM','geometry']]
# Rename the 'DISTRICTE' column to 'District'
bcn.rename(columns={'DISTRICTE': 'Districts'}, inplace=True)
# Create a new GeoDataFrame with the selected columns
bcn_gpd = gpd.GeoDataFrame(bcn)
# Add a new column 'District' with values 1 to 10
bcn_gpd['Districts'] = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
bcn_gpd

In [ ]:
# Merge Barcelona's districts map with votes in each district
merged_data = bcn_gpd.merge(income_bcn_district, on='Districts')
merged_data = merged_data.rename(columns={'Total': 'Average income per person/house (€)'})

merged_data

In [ ]:
# Plot the election results on a map of Barcelona's districts

fig = px.choropleth_mapbox(
    merged_data,
    geojson=merged_data.geometry,
    locations=merged_data.index,
    color='Average income per person/house (€)',  # Change to the column you want to visualize
    hover_name='Districts',
    mapbox_style="carto-positron",
    center={"lat": 41.3874, "lon": 2.1686},
    title= 'Income by district',# Center of Catalonia
    zoom=10
)

fig.update_layout(height=600, width=800)
fig.update_geos(fitbounds="locations", visible=False)
fig.show()